In [19]:
# Cell 1: Import necessary libraries
import os
import numpy as np
import pandas as pd
from scipy.signal import butter, filtfilt
from sklearn.model_selection import train_test_split
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv1D, MaxPooling1D, Flatten, Dense
from tensorflow.keras.optimizers import Adam
from sklearn.metrics import accuracy_score


In [26]:
# Cell 2: Define paths and parameters
input_data_path = "D:/AI/EEG_v2/data/input"
output_data_path = "D:/AI/EEG_v2/data/output"

# Parameters
sampling_rate = 100  # 100 Hz
epoch_length = 30 * sampling_rate  # 30 seconds
n_train = 454  # Use first 160 for training, remaining for validation


In [27]:
# Cell 3: Bandpass filter function
def bandpass_filter(signal, lowcut=0.5, highcut=30, fs=100, order=4):
    nyquist = 0.5 * fs
    low = lowcut / nyquist
    high = highcut / nyquist
    b, a = butter(order, [low, high], btype='band')
    return filtfilt(b, a, signal)


In [28]:
# Cell 4: Load and preprocess data function
def load_and_preprocess_data(input_path, output_path, n_patients):
    X, y = [], []
    for i in range(1, n_patients + 1):
        # Load input EEG signal
        eeg_file = os.path.join(input_path, f"subject_{i}.csv")
        output_file = os.path.join(output_path, f"subject_{i}.csv")
        
        # Load EEG and output (pain score)
        eeg_data = pd.read_csv(eeg_file, header=None).squeeze().values
        pain_score = pd.read_csv(output_file, header=None).squeeze().item()  # Corrected to use .item()
        
        # Apply bandpass filter
        eeg_filtered = bandpass_filter(eeg_data)
        
        # Segment into 30-second epochs
        epochs = [eeg_filtered[j:j+epoch_length] for j in range(0, len(eeg_filtered), epoch_length)]
        epochs = [epoch for epoch in epochs if len(epoch) == epoch_length]  # Filter incomplete epochs

        X.extend(epochs)
        y.extend([pain_score] * len(epochs))
        print(f'Patient {i} data loaded')
    
    return np.array(X), np.array(y)


In [29]:
# Cell 5: Load and preprocess data
X, y = load_and_preprocess_data(input_data_path, output_data_path, 568)

# Split data into train and validation sets
X_train, X_val, y_train, y_val = train_test_split(X, y, train_size=n_train, stratify=y)


Patient 1 data loaded
Patient 2 data loaded
Patient 3 data loaded
Patient 4 data loaded
Patient 5 data loaded
Patient 6 data loaded
Patient 7 data loaded
Patient 8 data loaded
Patient 9 data loaded
Patient 10 data loaded
Patient 11 data loaded
Patient 12 data loaded
Patient 13 data loaded
Patient 14 data loaded
Patient 15 data loaded
Patient 16 data loaded
Patient 17 data loaded
Patient 18 data loaded
Patient 19 data loaded
Patient 20 data loaded
Patient 21 data loaded
Patient 22 data loaded
Patient 23 data loaded
Patient 24 data loaded
Patient 25 data loaded
Patient 26 data loaded
Patient 27 data loaded
Patient 28 data loaded
Patient 29 data loaded
Patient 30 data loaded
Patient 31 data loaded
Patient 32 data loaded
Patient 33 data loaded
Patient 34 data loaded
Patient 35 data loaded
Patient 36 data loaded
Patient 37 data loaded
Patient 38 data loaded
Patient 39 data loaded
Patient 40 data loaded
Patient 41 data loaded
Patient 42 data loaded
Patient 43 data loaded
Patient 44 data load

In [35]:
def display_preprocessed_data(X, y, num_samples=5):
    print("Shape of X (epochs):", X.shape)
    print("Shape of y (pain scores):", y.shape)
    
    # Display a few sample epochs and their associated pain scores
    for i in range(min(num_samples, len(X))):
        print(f"\nSample {i + 1} - Pain Score: {y[i]}")
        print("Epoch Data:", X[i])
display_preprocessed_data(X, y)

Shape of X (epochs): (184446, 3000)
Shape of y (pain scores): (184446,)

Sample 1 - Pain Score: 0
Epoch Data: [-2.03257434e-08 -2.00767200e-08 -1.98061197e-08 ...  1.48874150e+01
  1.47843686e+01  1.46639936e+01]

Sample 2 - Pain Score: 0
Epoch Data: [  14.52599622   14.37009338   14.19601324 ...  -66.83416634  -77.22651686
 -118.8334868 ]

Sample 3 - Pain Score: 0
Epoch Data: [-116.29800947  -47.28860273    0.28009865 ...  -56.32949372 -103.27844477
 -104.54812917]

Sample 4 - Pain Score: 0
Epoch Data: [ -71.84881838  -85.0653995  -153.81425542 ... -180.4530775  -161.67551787
 -175.59272382]

Sample 5 - Pain Score: 0
Epoch Data: [-191.29142281 -194.54457643 -194.90515338 ...  -43.83793328  -43.72499164
  -55.12049039]


In [32]:
# Cell 6: Define 1D CNN model function
def build_1d_cnn(input_shape):
    model = Sequential([
        Conv1D(32, kernel_size=3, activation='relu', input_shape=input_shape),
        MaxPooling1D(pool_size=2),
        Conv1D(64, kernel_size=3, activation='relu'),
        MaxPooling1D(pool_size=2),
        Flatten(),
        Dense(128, activation='relu'),
        Dense(4, activation='softmax')  # 4 outputs for classes 0, 1, 2, 3
    ])
    model.compile(optimizer=Adam(learning_rate=0.001), loss='sparse_categorical_crossentropy', metrics=['accuracy'])
    return model


In [34]:
# Cell 7: Prepare data for CNN model
input_shape = (X_train.shape[1], 1)  # Shape of each epoch
X_train = X_train[..., np.newaxis]
X_val = X_val[..., np.newaxis]


In [36]:
# Cell 8: Build, train, and evaluate model
model = build_1d_cnn(input_shape)

history = model.fit(X_train, y_train, epochs=10, batch_size=32, validation_data=(X_val, y_val))


c:\Users\Admin\AppData\Local\Programs\Python\Python312\Lib\site-packages\keras\src\layers\convolutional\base_conv.py:107: UserWarning: Do not pass an `input_shape`/`input_dim` argument to a layer. When using Sequential models, prefer using an `Input(shape)` object as the first layer in the model instead.
  super().__init__(activity_regularizer=activity_regularizer, **kwargs)


Epoch 1/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 83s 6s/step - accuracy: 0.3434 - loss: 125.4580 - val_accuracy: 0.5093 - val_loss: 41.7059
Epoch 2/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 52s 4s/step - accuracy: 0.5016 - loss: 20.9048 - val_accuracy: 0.2055 - val_loss: 4.7234
Epoch 3/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 53s 4s/step - accuracy: 0.4466 - loss: 1.9826 - val_accuracy: 0.4824 - val_loss: 1.4718
Epoch 4/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 55s 4s/step - accuracy: 0.6947 - loss: 0.9272 - val_accuracy: 0.5037 - val_loss: 1.5162
Epoch 5/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 65s 5s/step - accuracy: 0.7115 - loss: 0.7334 - val_accuracy: 0.4652 - val_loss: 1.8136
Epoch 6/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 67s 5s/step - accuracy: 0.8103 - loss: 0.5915 - val_accuracy: 0.5004 - val_loss: 1.9686
Epoch 7/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 70s 5s/step - accuracy: 0.8416 - loss: 0.4256 - val_accuracy: 0.4615 - val_loss: 2.0311
Epoch 8/10
15/15 ━━━━━━━━━━━━━━━━━━━━ 75s 5s/step - accuracy: 0.9367 - loss: 0.2526 - val_accuracy: 0.4842 - val_l

In [37]:
# Evaluate on validation data
y_pred_probs = model.predict(X_val)  # Shape: (num_samples, 4)
y_pred_classes = np.argmax(y_pred_probs, axis=1)  # Shape: (num_samples,)
accuracy = accuracy_score(y_val, y_pred_classes)
print(f"Validation Accuracy: {accuracy * 100:.2f}%")

5750/5750 ━━━━━━━━━━━━━━━━━━━━ 50s 9ms/step
Validation Accuracy: 48.85%
